# Cleanup - OAM Component Contribution Demo

This notebook cleans up all resources created in `01_OAM-contrib.ipynb`.

## Resources to Clean Up

1. **KubeVela Application** - `my-dynamodb-app`
2. **Composite Resources** - `XDynamoDBTable` instances
3. **ComponentDefinition** - `simple-dynamodb`
4. **Composition** - `dynamodb-table.demo.kubecon.io`
5. **XRD** - `xdynamodbtables.demo.kubecon.io`
6. **Managed Resources** - DynamoDB Table resources

**Warning:** This will delete all resources created during the demo. Make sure you're ready before proceeding.

## Step 1: Check Current Resources

Let's see what resources currently exist.

In [1]:
%%bash
echo "=== Current Resources ==="
echo ""

echo "1. KubeVela Applications:"
kubectl get application -n default 2>/dev/null || echo "   None found"
echo ""

echo "2. Composite Resources (XDynamoDBTable):"
kubectl get xdynamodbtables -A 2>/dev/null || echo "   None found"
echo ""

echo "3. Managed DynamoDB Tables:"
kubectl get table -A 2>/dev/null || echo "   None found"
echo ""

echo "4. ComponentDefinition:"
kubectl get componentdefinition simple-dynamodb -n vela-system 2>/dev/null || echo "   None found"
echo ""

echo "5. Composition:"
kubectl get composition dynamodb-table.demo.kubecon.io 2>/dev/null || echo "   None found"
echo ""

echo "6. XRD:"
kubectl get xrd xdynamodbtables.demo.kubecon.io 2>/dev/null || echo "   None found"
echo ""

=== Current Resources ===

1. KubeVela Applications:
NAME              COMPONENT     TYPE         PHASE                HEALTHY   STATUS      AGE
product-catalog   product-api   webservice   workflowSuspending   true      Ready:1/1   8h

2. Composite Resources (XDynamoDBTable):
   None found

3. Managed DynamoDB Tables:

4. ComponentDefinition:
   None found

5. Composition:
   None found

6. XRD:
   None found



## Step 2: Delete KubeVela Application

Delete the KubeVela application if it exists.

In [2]:
%%bash
echo "Deleting KubeVela Application..."

if kubectl get application my-dynamodb-app -n default &>/dev/null; then
    kubectl delete application my-dynamodb-app -n default
    echo "✓ Application 'my-dynamodb-app' deleted"
else
    echo "⚠ Application not found (may already be deleted)"
fi

echo ""
echo "Remaining applications:"
kubectl get application -n default 2>/dev/null || echo "None"

Deleting KubeVela Application...
⚠ Application not found (may already be deleted)

Remaining applications:
NAME              COMPONENT     TYPE         PHASE                HEALTHY   STATUS      AGE
product-catalog   product-api   webservice   workflowSuspending   true      Ready:1/1   8h


## Step 3: Delete Composite Resources

Delete all XDynamoDBTable composite resources.

In [3]:
%%bash
echo "Deleting Composite Resources..."

# Delete specific test resource
if kubectl get xdynamodbtables test-simple-table -n default &>/dev/null; then
    kubectl delete xdynamodbtables test-simple-table -n default
    echo "✓ Composite resource 'test-simple-table' deleted"
else
    echo "⚠ test-simple-table not found"
fi

echo ""
echo "Checking for other composite resources..."
RESOURCES=$(kubectl get xdynamodbtables -A --no-headers 2>/dev/null | wc -l)

if [ "$RESOURCES" -gt 0 ]; then
    echo "Found $RESOURCES additional resource(s). Deleting all..."
    kubectl delete xdynamodbtables -n default --all
    echo "✓ All composite resources deleted"
else
    echo "✓ No additional resources found"
fi

Deleting Composite Resources...
⚠ test-simple-table not found

Checking for other composite resources...
✓ No additional resources found


## Step 4: Wait for Managed Resources Cleanup

When composite resources are deleted, Crossplane will clean up the underlying managed resources. Let's wait for this to complete.

In [4]:
%%bash
echo "Waiting for managed resources cleanup..."
echo "(This ensures DynamoDB tables and other managed resources are deleted)"
echo ""

for i in {1..10}; do
    TABLES=$(kubectl get table -A --no-headers 2>/dev/null | wc -l)
    
    if [ "$TABLES" -eq 0 ]; then
        echo "✓ All managed resources cleaned up"
        break
    fi
    
    echo "  Waiting... ($i/10) - $TABLES table(s) still being deleted"
    sleep 3
done

echo ""
echo "Current managed tables:"
kubectl get table -A 2>/dev/null || echo "None"

Waiting for managed resources cleanup...
(This ensures DynamoDB tables and other managed resources are deleted)

✓ All managed resources cleaned up

Current managed tables:
None


## Step 5: Delete ComponentDefinition

Delete the KubeVela ComponentDefinition.

In [5]:
%%bash
echo "Deleting ComponentDefinition..."

if kubectl get componentdefinition simple-dynamodb -n vela-system &>/dev/null; then
    kubectl delete componentdefinition simple-dynamodb -n vela-system
    echo "✓ ComponentDefinition 'simple-dynamodb' deleted"
else
    echo "⚠ ComponentDefinition not found (may not have been installed)"
fi

echo ""
echo "Remaining ComponentDefinitions:"
kubectl get componentdefinition -n vela-system | grep dynamodb || echo "None"

Deleting ComponentDefinition...
⚠ ComponentDefinition not found (may not have been installed)

Remaining ComponentDefinitions:


E1105 21:46:53.238215   74445 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:53.240381   74445 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:53.242221   74445 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:53.243916   74445 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:53.245811   74445 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:808

None


## Step 6: Delete Composition

Delete the Crossplane Composition.

In [6]:
%%bash
echo "Deleting Composition..."

if kubectl get composition dynamodb-table.demo.kubecon.io &>/dev/null; then
    kubectl delete composition dynamodb-table.demo.kubecon.io
    echo "✓ Composition 'dynamodb-table.demo.kubecon.io' deleted"
else
    echo "⚠ Composition not found"
fi

echo ""
echo "Remaining Compositions:"
kubectl get composition | grep demo.kubecon.io || echo "None"

Deleting Composition...
⚠ Composition not found

Remaining Compositions:


E1105 21:46:56.040456   74455 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:56.043071   74455 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:56.045424   74455 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:56.047377   74455 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:56.049224   74455 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:808

None


## Step 7: Delete XRD (CompositeResourceDefinition)

Delete the Crossplane XRD. This should be done last after all composite resources are gone.

In [7]:
%%bash
echo "Deleting XRD..."

if kubectl get xrd xdynamodbtables.demo.kubecon.io &>/dev/null; then
    kubectl delete xrd xdynamodbtables.demo.kubecon.io
    echo "✓ XRD 'xdynamodbtables.demo.kubecon.io' deleted"
else
    echo "⚠ XRD not found"
fi

echo ""
echo "Remaining XRDs:"
kubectl get xrd | grep demo.kubecon.io || echo "None"

Deleting XRD...
⚠ XRD not found

Remaining XRDs:


E1105 21:46:59.107676   74464 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:59.110108   74464 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:59.112181   74464 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:59.114324   74464 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:8080/api?timeout=32s\": dial tcp 127.0.0.1:8080: connect: connection refused"
E1105 21:46:59.116287   74464 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: Get \"http://localhost:808

None


## Step 8: Verification

Verify all resources have been cleaned up successfully.

In [8]:
%%bash
echo "=========================================="
echo "Cleanup Verification"
echo "=========================================="
echo ""

ALL_CLEAN=true

# Check Applications
if kubectl get application my-dynamodb-app -n default &>/dev/null; then
    echo "✗ Application still exists"
    ALL_CLEAN=false
else
    echo "✓ Application removed"
fi

# Check Composite Resources
COMPOSITES=$(kubectl get xdynamodbtables -A --no-headers 2>/dev/null | wc -l)
if [ "$COMPOSITES" -eq 0 ]; then
    echo "✓ Composite resources removed"
else
    echo "✗ $COMPOSITES composite resource(s) still exist"
    ALL_CLEAN=false
fi

# Check Managed Resources
TABLES=$(kubectl get table -A --no-headers 2>/dev/null | wc -l)
if [ "$TABLES" -eq 0 ]; then
    echo "✓ Managed resources removed"
else
    echo "⚠ $TABLES managed table(s) still being deleted"
fi

# Check ComponentDefinition
if kubectl get componentdefinition simple-dynamodb -n vela-system &>/dev/null; then
    echo "✗ ComponentDefinition still exists"
    ALL_CLEAN=false
else
    echo "✓ ComponentDefinition removed"
fi

# Check Composition
if kubectl get composition dynamodb-table.demo.kubecon.io &>/dev/null; then
    echo "✗ Composition still exists"
    ALL_CLEAN=false
else
    echo "✓ Composition removed"
fi

# Check XRD
if kubectl get xrd xdynamodbtables.demo.kubecon.io &>/dev/null; then
    echo "✗ XRD still exists"
    ALL_CLEAN=false
else
    echo "✓ XRD removed"
fi

echo ""
echo "=========================================="
if [ "$ALL_CLEAN" = true ]; then
    echo "✓ Cleanup Complete!"
    echo "All resources have been successfully removed."
else
    echo "⚠ Cleanup Incomplete"
    echo "Some resources still exist. You may need to:"
    echo "  1. Wait longer for managed resources to finish deleting"
    echo "  2. Manually delete resources that failed to remove"
    echo "  3. Check for finalizers preventing deletion"
fi
echo "=========================================="

Cleanup Verification

✓ Application removed
✓ Composite resources removed
✓ Managed resources removed
✓ ComponentDefinition removed
✓ Composition removed
✓ XRD removed

✓ Cleanup Complete!
All resources have been successfully removed.


## Step 9: Clean Up Local Files (Optional)

The cleanup above only removes Kubernetes resources. Local YAML/CUE files are preserved.

**Warning:** This will delete the local files created during the demo.

In [9]:
%%bash
echo "Local files in current directory:"
echo ""
ls -la crossplane/ kubevela/ test/ 2>/dev/null || echo "No demo directories found"
echo ""
echo "To remove local files, uncomment and run the following:"
echo "# rm -rf crossplane kubevela test"

Local files in current directory:

crossplane/:
total 0
drwxr-xr-x@  3 jguionnet  staff   96 Oct 23 20:40 .
drwxr-xr-x@ 21 jguionnet  staff  672 Nov  5 12:25 ..
drwxr-xr-x@  4 jguionnet  staff  128 Oct 24 15:59 dynamodb

kubevela/:
total 0
drwxr-xr-x@  3 jguionnet  staff   96 Oct 23 20:42 .
drwxr-xr-x@ 21 jguionnet  staff  672 Nov  5 12:25 ..
drwxr-xr-x@  3 jguionnet  staff   96 Oct 23 20:42 components

test/:
total 24
drwxr-xr-x@  5 jguionnet  staff   160 Oct 27 18:05 .
drwxr-xr-x@ 21 jguionnet  staff   672 Nov  5 12:25 ..
-rw-r--r--@  1 jguionnet  staff   339 Nov  5 12:24 app.yaml
-rw-r--r--@  1 jguionnet  staff  1829 Oct 28 15:52 app2.yaml
-rw-r--r--@  1 jguionnet  staff   698 Nov  5 12:24 test-table.yaml

To remove local files, uncomment and run the following:
# rm -rf crossplane kubevela test


## Summary

This notebook cleaned up all resources created in the OAM Component Contribution demo:

✓ KubeVela Applications deleted  
✓ Composite Resources (XDynamoDBTable) deleted  
✓ Managed Resources (DynamoDB Tables) cleaned up  
✓ ComponentDefinition removed  
✓ Composition removed  
✓ XRD removed  

### Cleanup Order

The cleanup follows the reverse order of creation:
1. Applications (highest level)
2. Composite Resources
3. Managed Resources (automatic)
4. ComponentDefinition
5. Composition
6. XRD (lowest level)

### Preserved Files

Local files in these directories are preserved:
- `crossplane/dynamodb/` - XRD and Composition YAML files
- `kubevela/components/dynamodb/` - CUE files
- `test/` - Test resources

You can re-run the demo using these files without recreating them.